<h2> Taller 4 - CoppeliaSim y Python</h2>

<br>Requiere que los archivos 'sim.py', 'simConst.py', 'remoteapi.dll' estén alojados en la misma carpeta que este cuaderno de Jupyter
<br><h4> Instrucciones</h4>
<br>1. Abra la escena robotPlanar.ttt
<br>2. Seleccionando la base del robot desde la jerarquía de escena, presione botón derecho del mouse y agregue un archivo de script mediante Add -> Associated child script -> Non threaded. Aparecerá un pequeño ícono de documento junto al nombre del robot en la escena de jerarquía.
<br>3. En el script es posible incluir código de programación, escritos en lenguaje LUA. Para nuestro caso, todo el código que requeriremos es habilitar el API remoto, asignando un puerto de comunicación. En la función <b>sysCall_init()</b> agregue la siguiente línea:
<br> 
<font face = "Monospace"> <pre> simRemoteApi.start(19999)</font>
<br> Proceda a continuación con las actividades:

In [112]:
import sim
import numpy as np

### 1. Establecer la conexión
Utilizaremos las funciones del API Remoto de VREP.
<br> Para más detalles refiérase a la documentación de la librería:
<br> http://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctionsPython.htm

In [113]:
def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [114]:
# Conectarse al servidor de VREP
# *** ejecutar cada vez que se reinicia la simulación ***
clientID = connect(19999)

conectado a 19999


<h3> 2. Obtener los manejadores (handlers)</h3>
Un manejador (handler) es un número identificador que asigna VREP para cada uno de los elementos de la escena. A través de su manejador se puede hacer referencia a un elemento en específico de la escena.

In [115]:
# Obtenemos el manejador para el dummy 
returnCode,handle=sim.simxGetObjectHandle(clientID,'dummy',sim.simx_opmode_blocking)
dummy = handle
print(dummy)

21


In [116]:
# A partir de su manejador podemos accionar sobre el objeto,
# por ejemplo, obtener su posición
returnCode,pos=sim.simxGetObjectPosition(clientID, dummy, -1, sim.simx_opmode_blocking)
print(pos)

[0.4999404549598694, 0.004880063701421022, 8.853242405848505e-08]


In [117]:
# Obtenemos los manejadores para cada una de las articulaciones del robot
ret,joint1=sim.simxGetObjectHandle(clientID,'joint1',sim.simx_opmode_blocking)
ret,joint2=sim.simxGetObjectHandle(clientID,'joint2',sim.simx_opmode_blocking)
print(joint1, joint2)

17 19


<h3> 3. ¿Cuál es la posición de las articulaciones?</h3>
Utilizando los manejadores, podemos obtener información de los elementos.

In [118]:
# leemos la posición de joint1, en radianes.
returnCode, pos1 = sim.simxGetJointPosition(clientID, joint1, sim.simx_opmode_blocking)
print(pos1)

-1.9257584416010332e-10


In [119]:
returnCode, pos2 = sim.simxGetJointPosition(clientID, joint2, sim.simx_opmode_blocking)
print(pos2)

0.024402718991041183


<h3> 4. ... y movemos el robot</h3>
Utilizando los manejadores, podemos enviar parámetros a los elementos.

In [135]:
# enviamos la posición de joint1, en radianes.
q1 = 0 * np.pi/180
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, q1, sim.simx_opmode_oneshot)
print(returnCode)

0


In [132]:
q2 = 40 * np.pi/180
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, q2, sim.simx_opmode_oneshot)
print(returnCode)

0


In [122]:
# Ponemos todo el código junto
# conectamos
clientID = connect(19999)
# obtenemos los manejadores
returnCode,handle=sim.simxGetObjectHandle(clientID,'dummy',sim.simx_opmode_blocking)
dummy = handle
ret,joint1=sim.simxGetObjectHandle(clientID,'joint1',sim.simx_opmode_blocking)
ret,joint2=sim.simxGetObjectHandle(clientID,'joint2',sim.simx_opmode_blocking)
#movemos
q1 = -90 * np.pi/180
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, q1, sim.simx_opmode_oneshot)
q2 = 0 * np.pi/180
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, q2, sim.simx_opmode_oneshot)

conectado a 19999


In [123]:
returnCode,pos=sim.simxGetObjectPosition(clientID, dummy, -1, sim.simx_opmode_blocking)
print(pos)

[0.41895967721939087, -0.028877634555101395, 8.731550593665816e-08]


Código para saber si esta todo correcto

In [124]:
returnCode, handle = sim.simxGetObjectHandle(clientID, 'dummy', sim.simx_opmode_blocking)
if returnCode != sim.simx_return_ok:
    print("Error al obtener el handle de 'dummy'")
    exit()

ret, joint1 = sim.simxGetObjectHandle(clientID, 'joint1', sim.simx_opmode_blocking)
if ret != sim.simx_return_ok:
    print("Error al obtener el handle de 'joint1'")
    exit()

ret, joint2 = sim.simxGetObjectHandle(clientID, 'joint2', sim.simx_opmode_blocking)
if ret != sim.simx_return_ok:
    print("Error al obtener el handle de 'joint2'")
    exit()

Código completo con cambios para que se mueva y vuelva al principio

In [ ]:
import time
import numpy as np

# Verificar conexión con CoppeliaSim
if clientID != -1:
    print("Conexión establecida correctamente.")
else:
    print("Error en la conexión con CoppeliaSim.")
    exit()

# Enviar los comandos de movimiento con opmode_blocking
q1 = 0 * np.pi / 180  # Ángulo para joint1 (cambiar a un valor simple)
q2 = 0 * np.pi / 180  # Ángulo para joint2

# Mover joint1
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, q1, sim.simx_opmode_blocking)
if returnCode != sim.simx_return_ok:
    print("Error al mover joint1")
else:
    print("Movimiento de joint1 solicitado")

# Mover joint2
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, q2, sim.simx_opmode_blocking)
if returnCode != sim.simx_return_ok:
    print("Error al mover joint2")
else:
    print("Movimiento de joint2 solicitado")

# Esperar un segundo para que los movimientos se apliquen
time.sleep(1)

# Verificar las posiciones actuales de las articulaciones
ret, jointPos1 = sim.simxGetJointPosition(clientID, joint1, sim.simx_opmode_blocking)
ret, jointPos2 = sim.simxGetJointPosition(clientID, joint2, sim.simx_opmode_blocking)

# Mostrar las posiciones de las articulaciones
print(f"Posición actual de joint1: {jointPos1}")
print(f"Posición actual de joint2: {jointPos2}")

Conexión establecida correctamente.
Movimiento de joint1 solicitado
Movimiento de joint2 solicitado
Posición actual de joint1: -7.974923832421155e-09
Posición actual de joint2: 0.02335146628320217
